In [1]:
import pandas as pd
import numpy as np
import datetime
import sys, os
import glob
import time
import fnmatch
import os
import datetime
import xlwings as xw
from datetime import date
from datetime import timedelta
start_time = time.time()

a=fnmatch.filter(os.listdir('.'), 'ZCM DAILY*')
a=a[0]

july1 = datetime.date(2022, 7, 3)
strings = date.today() + timedelta(days=1)
strings = str(strings)

print('Reading input', a)

Reading input ZCM DAILY 7.11.xlsx


# OH  TX2 DAILY

In [2]:
zcmtx2 = pd.read_excel(a,dtype={'*Zebra PN': str, '*Zebra Site': str,'*Supplier ID': str}, sheet_name='TX2')

zcmtx2['*Date'] = pd.to_datetime(zcmtx2['#SCHEDULE_DOCK_DATE']).dt.date
zcmtx2.head()
zcmtx2.drop(labels=['#ATTRIBUTE1','#ATTRIBUTE2','#ATTRIBUTE3'], axis=1, inplace = True)
zcmtx2.rename(columns={'#QUANTITY': 'Quantity'},inplace=True)
#get date of june 30 2022
july1 = datetime.date(2022, 7, 3)

#identify infor before july
mask1 = (zcmtx2['*Date'] <= july1)
zcmtx2['month'] = np.where(mask1, 'beforejuly','afterjuly')

#zcmuploads = zcmuploads[(zcmuploads['*Transaction Type'] == 'ONHANDWK') | (zcmuploads['*Transaction Type'] == 'Net Requirements') | 
                       #(zcmuploads['*Transaction Type'] == 'GROSSFRCTWK') | 
                       #(zcmuploads['*Transaction Type'] == 'ONHANDDLY')] #delete extra rows

zcmohtx2= zcmtx2[(zcmtx2['#TRANSACTION_TYPE']=='ONHANDDLY')]
internalreqtx2= zcmtx2[(zcmtx2['#TRANSACTION_TYPE']=='INTERNALREQDLY')]
sales_orders_tx2= zcmtx2[(zcmtx2['#TRANSACTION_TYPE']=='BOOKEDSODLY')]#so #new Ty change
#sales_orders_tx2.drop(labels=['month'], axis=1, inplace = True)

sales_orders_tx2

#zcmtx2.head()
#zcmohtx2
#internalreqtx2

netreqtx2 = pd.read_excel(a,dtype={'#Customer Item Name': str, '#Site Name': str,'#Supplier': str}, sheet_name='NET REQ TX2')

netreqtx2['*Date'] = pd.to_datetime(netreqtx2['#Date']).dt.date
#identify infor before july
mask1 = (netreqtx2['*Date'] <= july1)
netreqtx2['month'] = np.where(mask1, 'beforejuly','afterjuly')

#zcmuploads = zcmuploads[(zcmuploads['*Transaction Type'] == 'ONHANDWK') | (zcmuploads['*Transaction Type'] == 'Net Requirements') | 
                       #(zcmuploads['*Transaction Type'] == 'GROSSFRCTWK') | 
                       #(zcmuploads['*Transaction Type'] == 'ONHANDDLY')] #delete extra rows

netreqtx2.drop(labels=['#FlexAttr_String_PIT_01','#FlexAttr_String_PIT_02','#FlexAttr_String_PIT_03','#Supplier Item Name'], axis=1, inplace = True)
netreqtx2.rename(columns={'#Quantity': 'Quantity'},inplace=True)
netreqtx2.rename(columns={'#Customer Item Name': '*Zebra PN'},inplace=True)
netreqtx2.rename(columns={'#Supplier': '*Supplier ID'},inplace=True)
netreqtx2.head()

netreqtx2= netreqtx2[(netreqtx2['month']=='beforejuly')]  
netreqtx2.head()

#sum of oh
zcmohtx2_piv=zcmohtx2.pivot_table(index=('*Zebra PN','*Supplier ID'), columns='month', aggfunc='sum').fillna(0)
#print(len(zcmohtx2_piv))
#zcmuploadspiv['*Transaction Type'] ='ONHANDWK'
zcmohtx2_piv.head()

#sum of NET REQ
netreqtx2_piv=netreqtx2.pivot_table(index=('*Zebra PN','*Supplier ID'), columns='month', aggfunc='sum').fillna(0)
#print(len(netreqtx2_piv))
#zcmuploadspiv['*Transaction Type'] ='ONHANDWK'
netreqtx2_piv.head()

#sum of IRS TO SUM TO fcst
internalreqtx2_piv=internalreqtx2.pivot_table(index=('*Zebra PN','*Supplier ID'), columns='month', aggfunc='sum').fillna(0)
#print(len(internalreqtx2_piv))
#zcmuploadspiv['*Transaction Type'] ='ONHANDWK'
internalreqtx2_piv.head()

#assign corresponding NET REQ TO rows that will be uploaded
zcmohtx2_0 = pd.merge(zcmohtx2_piv,netreqtx2_piv, on =['*Zebra PN','*Supplier ID'],how='left')
zcmohtx2_0

#to oh and net req add IRS
zcmohtx2_1 = pd.merge(zcmohtx2_0,internalreqtx2_piv, on =['*Zebra PN','*Supplier ID'],how='left')
zcmohtx2_1

#DROP LEVELS AND merge their names
zcmohtx2_1.columns = ['_'.join(col) for col in zcmohtx2_1.columns.values]
zcmohtx2_1.reset_index(inplace=True)
zcmohtx2_1

#rename columns
zcmohtx2_1.rename(columns={'Quantity_x_afterjuly': 'currentOH'},inplace=True)
#zcmohtx2_1['Net_Req_bf_july'].fillna(0, inplace=True) #change 7/4

#zcmohtx2_1['IR_aft_july'].fillna(0, inplace=True) #change 7/4
#zcmohtx2_1['IR_bf_july'].fillna(0, inplace=True)
zcmohtx2_1

#sum of SO
sales_orders_tx2_piv=sales_orders_tx2.pivot_table(index=('*Zebra PN','*Supplier ID'), columns= 'month', aggfunc='sum').fillna(0)
#print(len(sales_orders_tx2_piv))
#zcmuploadspiv['*Transaction Type'] ='ONHANDWK'
sales_orders_tx2_piv.head()

zcmohtx2_1 = pd.merge(zcmohtx2_1, sales_orders_tx2_piv, on =['*Zebra PN','*Supplier ID'],how='left')
zcmohtx2_1

zcmohtx2_1[('Quantity', 'afterjuly')].fillna(0, inplace=True) #SO
#zcmohtx2_1[('Quantity', 'beforejuly')].fillna(0, inplace=True) #SO
zcmohtx2_1

#COMPARE SO QTY against NET REQ QTY, KEEP BIGGER FOR TOTAL DEMAND CALCULATION
#zcmohtx2_1['Net_Req_bf_july'] = np.where(zcmohtx2_1['Net_Req_bf_july'] < zcmohtx2_1[('Quantity', 'beforejuly')],
 #                                       zcmohtx2_1[('Quantity', 'beforejuly')], zcmohtx2_1['Net_Req_bf_july'])



zcmohtx2_1['Total_demand']= zcmohtx2_1[('Quantity', 'afterjuly')]
zcmohtx2_1

#if OH less than total demand before july, then put all oh in TX2
zcmohtx2_1['*Inventory Level']=  np.where(zcmohtx2_1['Total_demand'] < zcmohtx2_1['currentOH'], 
                                         zcmohtx2_1['Total_demand'],zcmohtx2_1['currentOH'])
zcmohtx2_1.head()

#SET OH AS 0 for PN with no demand beforejuly
zcmohtx2_1['*Inventory Level']=  np.where(zcmohtx2_1['Total_demand'] == 0,0,zcmohtx2_1['*Inventory Level'])
zcmohtx2_1['*Inventory Level']=zcmohtx2_1['*Inventory Level'].fillna(0)
zcmohtx2_1.head()
zcmohtx2_1

pto_kit_exclude = pd.read_excel(a,dtype={'PART_NUMBER': str}, sheet_name='Combined List Pivot',header=1)
pto_kit_exclude = pto_kit_exclude[['PART_NUMBER']]
pto_kit_exclude['PTO_KIT'] = 'PTO'
pto_kit_exclude.rename(columns={'PART_NUMBER': '*Zebra PN'},inplace=True)
pto_kit_exclude.head()

zcmohtx2_11 = pd.merge(zcmohtx2_1,pto_kit_exclude, on =['*Zebra PN'],how='left')
zcmohtx2_11['PTO_KIT'].fillna("NO PTO", inplace=True)
zcmohtx2_11

zcmohtx2_11['*Inventory Level']=  np.where(zcmohtx2_11['PTO_KIT'] == 'PTO',
                                           zcmohtx2_11['currentOH'],zcmohtx2_11['*Inventory Level'])
zcmohtx2_11

#drop extra columns
zcmohtx2_2 = zcmohtx2_11[['*Zebra PN','*Supplier ID','*Inventory Level']]
zcmohtx2_2['*Transaction Type'] = 'ONHANDDLY'
zcmohtx2_2['*Extraction Date'] = date.today() + timedelta(days=1)
zcmohtx2_2

zcm_other_measurestx2= zcmtx2[(zcmtx2['#TRANSACTION_TYPE']!='ONHANDDLY')]
zcm_other_measurestx2 = zcm_other_measurestx2[['*Zebra PN','*Supplier ID','Quantity','#TRANSACTION_TYPE','*Date']]
zcm_other_measurestx2.rename(columns={'Quantity': '*Inventory Level','#TRANSACTION_TYPE':'*Transaction Type',
                                     '*Date':'*Extraction Date'},inplace=True)

zcmohtx2_3 = zcmohtx2_2.append(zcm_other_measurestx2, ignore_index=True)
zcmohtx2_3.insert(0, '#*Zebra', 'Zebra')
mid = zcmohtx2_3['*Supplier ID']
zcmohtx2_3.drop(labels=['*Supplier ID'], axis=1, inplace = True)
zcmohtx2_3.insert(1, '*Supplier ID', mid)
zcmohtx2_3.insert(3,'*Zebra Site','TX2')
zcmohtx2_3.insert(4,'Location Code','')
mid = zcmohtx2_3['*Extraction Date']
zcmohtx2_3.drop(labels=['*Extraction Date'], axis=1, inplace = True)
zcmohtx2_3.insert(6, '*Extraction Date', mid)
zcmohtx2_3['Flex String PIT 1']=''
zcmohtx2_3['Flex String PIT 2']=''
zcmohtx2_3['Flex String PIT 3'] = ''
zcmohtx2_3['*Extraction Date'] = zcmohtx2_3['*Extraction Date'].astype(str)
zcmohtx2_3['*Extraction Date'] = zcmohtx2_3['*Extraction Date'] + 'T00:00:00+0000'
zcmohtx2_3

zcmohtx2_3= zcmohtx2_3[(zcmohtx2_3['*Transaction Type']!='BOOKEDSODLY')]
#print(zcmohtx2_3.head())

strings = date.today() + timedelta(days=1)
strings = str(strings)
stringstx2R= 'OH_TX2_DAILY_' + strings 
b=("%s.xlsx" % stringstx2R)

with pd.ExcelWriter(b) as writer:  
       zcmohtx2_3.to_excel(writer, index= False)

###validation TX2
#strings = date.today() + timedelta(days=1)
#strings = str(strings)
#stringstx2= 'OH_TX2_DAILY_Validation' + strings 
#b=("%s.xlsx" % stringstx2)
               #zcmohtx2_3.to_csv(r'c:\data\pandas.txt', header=None, index=None, sep=' ', mode='a')
#with pd.ExcelWriter(b) as writer:  
 #      zcmohtx2_11.to_excel(writer, index= False)

#zcmohtx2_3.loc[0] = zcmohtx2_3.columns #fix to add instead of re-write
#zcmohtx2_3

#headers = ['#0', 1, 2,3,4,5,6,7,8,9,10] 
#zcmohtx2_3.columns = headers
#zcmohtx2_3

#strings = date.today() + timedelta(days=1)
#strings
#strings = str(strings)
#strings = strings.replace('-','')
#strings
#stringstx2R= 'ssp-purchasediteminventory-1.0_TX2_ONHANDDLY_' + strings 
#b=("%s.xls" % stringstx2R)

#with pd.ExcelWriter(b) as writer:  
 #      zcmohtx2_3.to_excel(writer, index= False,sheet_name='PurchasedItemInventory')

#add until 4th september OH TX3
sept = datetime.date(2022, 9, 3)
sept

zcmtx3 = pd.read_excel(a,dtype={'*Zebra PN': str, '*Zebra Site': str,'*Supplier ID': str}, sheet_name='TX3')

zcmtx3['*Date'] = pd.to_datetime(zcmtx3['#SCHEDULE_DOCK_DATE']).dt.date
zcmtx3.head()

#identify infor before july
mask1 = (zcmtx3['*Date'] <= sept)
zcmtx3['month'] = np.where(mask1, 'beforejuly','afterjuly')

zcmtx3.drop(labels=['#ATTRIBUTE1','#ATTRIBUTE2','#ATTRIBUTE3'], axis=1, inplace = True)
zcmtx3.rename(columns={'#QUANTITY': 'Quantity'},inplace=True)

zcmohtx3= zcmtx3[(zcmtx3['#TRANSACTION_TYPE']=='ONHANDDLY')]
internalreqtx3= zcmtx3[(zcmtx3['#TRANSACTION_TYPE']=='INTERNALREQDLY')]

sales_orders_tx3= zcmtx3[(zcmtx3['#TRANSACTION_TYPE']=='BOOKEDSODLY')]#so #ty change consider all so
#sales_orders_tx3.drop(labels=['month'], axis=1, inplace = True)

#shared on prev sunday by Hari, as this are updated weekly, not by supply chain
netreqtx3 = pd.read_excel(a,dtype={'#Customer Item Name': str, '#Site Name': str,'#Supplier': str}, sheet_name='NET REQ TX3')

netreqtx3['*Date'] = pd.to_datetime(netreqtx3['#Date']).dt.date
#identify infor before july
mask1 = (netreqtx3['*Date'] <= sept)
netreqtx3['month'] = np.where(mask1, 'beforejuly','afterjuly')

#zcmuploads = zcmuploads[(zcmuploads['*Transaction Type'] == 'ONHANDWK') | (zcmuploads['*Transaction Type'] == 'Net Requirements') | 
                       #(zcmuploads['*Transaction Type'] == 'GROSSFRCTWK') | 
                       #(zcmuploads['*Transaction Type'] == 'ONHANDDLY')] #delete extra rows

netreqtx3.drop(labels=['#FlexAttr_String_PIT_01','#FlexAttr_String_PIT_02','#FlexAttr_String_PIT_03','#Supplier Item Name'], axis=1, inplace = True)
netreqtx3.rename(columns={'#Quantity': 'Quantity'},inplace=True)
netreqtx3.rename(columns={'#Customer Item Name': '*Zebra PN'},inplace=True)
netreqtx3.rename(columns={'#Supplier': '*Supplier ID'},inplace=True)
netreqtx3.head()

netreqtx3= netreqtx3[(netreqtx3['month']=='beforejuly')]
netreqtx3.head()

#sum of oh
zcmohtx3_piv=zcmohtx3.pivot_table(index=('*Zebra PN','*Supplier ID'), columns='month', aggfunc='sum').fillna(0)
#print(len(zcmohtx3_piv))
#zcmuploadspiv['*Transaction Type'] ='ONHANDWK'
zcmohtx3_piv.head()

#sum of NET REQ
netreqtx3_piv=netreqtx3.pivot_table(index=('*Zebra PN','*Supplier ID'), columns='month', aggfunc='sum').fillna(0)
#print(len(netreqtx3_piv))
#zcmuploadspiv['*Transaction Type'] ='ONHANDWK'
netreqtx3_piv.head()

#sum of IRS TO SUM TO fcst
internalreqtx3_piv=internalreqtx3.pivot_table(index=('*Zebra PN','*Supplier ID'), columns='month', aggfunc='sum').fillna(0)
#print(len(internalreqtx3_piv))
#zcmuploadspiv['*Transaction Type'] ='ONHANDWK'
internalreqtx3_piv.head()

#assign corresponding NET REQ TO rows that will be uploaded
zcmohtx3_0 = pd.merge(zcmohtx3_piv,netreqtx3_piv, on =['*Zebra PN','*Supplier ID'],how='left')
zcmohtx3_0

#to oh and net req add IRS
zcmohtx3_1 = pd.merge(zcmohtx3_0,internalreqtx3_piv, on =['*Zebra PN','*Supplier ID'],how='left')
zcmohtx3_1

#DROP LEVELS AND merge their names
zcmohtx3_1.columns = ['_'.join(col) for col in zcmohtx3_1.columns.values]
zcmohtx3_1.reset_index(inplace=True)
zcmohtx3_1

#rename columns
zcmohtx3_1.rename(columns={'Quantity_x_beforejuly': 'currentOH','Quantity_y_beforejuly':'Net_Req_bf_july',
                          'Quantity_afterjuly':'IR_aft_july','Quantity_beforejuly':'IR_bf_july'},inplace=True)

zcmohtx3_1['Net_Req_bf_july'].fillna(0, inplace=True)
#zcmohtx3_1['IR_aft_july'].fillna(0, inplace=True)
#zcmohtx3_1['IR_bf_july'].fillna(0, inplace=True)

#sum of SO
sales_orders_tx3_piv=sales_orders_tx3.pivot_table(index=('*Zebra PN','*Supplier ID'),columns= 'month', aggfunc='sum').fillna(0)
#print(len(sales_orders_tx3_piv))
#zcmuploadspiv['*Transaction Type'] ='ONHANDWK'
sales_orders_tx3_piv.head()

zcmohtx3_1 = pd.merge(zcmohtx3_1, sales_orders_tx3_piv, on =['*Zebra PN','*Supplier ID'],how='left')
zcmohtx3_1[('Quantity', 'beforejuly')].fillna(0, inplace=True)
zcmohtx3_1[('Quantity', 'afterjuly')].fillna(0, inplace=True)

zcmohtx3_1['Net_Req_bf_july'] = np.where(zcmohtx3_1['Net_Req_bf_july'] < zcmohtx3_1[('Quantity', 'beforejuly')],
                                         zcmohtx3_1[('Quantity', 'beforejuly')], zcmohtx3_1['Net_Req_bf_july'])

#zcmohtx2_1['Total_demand']=zcmohtx2_1['Net_Req_bf_july'] + zcmohtx2_1[('Quantity', 'afterjuly')]

zcmohtx3_1['Total_demand']=zcmohtx3_1['Net_Req_bf_july'] + zcmohtx3_1[('Quantity', 'afterjuly')]
#if OH less than total demand before july, then put all oh in TX3
zcmohtx3_1['*Inventory Level']=  np.where(zcmohtx3_1['Total_demand'] < zcmohtx3_1['currentOH'], 
                                         zcmohtx3_1['Total_demand'],zcmohtx3_1['currentOH'])
zcmohtx3_1.head()

#SET OH AS 0 for PN with no demand beforejuly
zcmohtx3_1['*Inventory Level']=  np.where(zcmohtx3_1['Total_demand'] == 0,0,zcmohtx3_1['*Inventory Level'])
zcmohtx3_1['*Inventory Level']=zcmohtx3_1['*Inventory Level'].fillna(0)
zcmohtx3_1.head()
zcmohtx3_1

zcmohtx3_2 = zcmohtx3_1[['*Zebra PN','*Supplier ID','*Inventory Level']]
zcmohtx3_2['*Transaction Type'] = 'ONHANDDLY'
zcmohtx3_2['*Extraction Date'] = date.today() + timedelta(days=1)
zcmohtx3_2

zcm_other_measurestx3= zcmtx3[(zcmtx3['#TRANSACTION_TYPE']!='ONHANDDLY')]
zcm_other_measurestx3 = zcm_other_measurestx3[['*Zebra PN','*Supplier ID','Quantity','#TRANSACTION_TYPE','*Date']]
zcm_other_measurestx3.rename(columns={'Quantity': '*Inventory Level','#TRANSACTION_TYPE':'*Transaction Type',
                                     '*Date':'*Extraction Date'},inplace=True)

zcmohtx3_3 = zcmohtx3_2.append(zcm_other_measurestx3, ignore_index=True)
zcmohtx3_3.insert(0, '#*Zebra', 'Zebra')
mid = zcmohtx3_3['*Supplier ID']
zcmohtx3_3.drop(labels=['*Supplier ID'], axis=1, inplace = True)
zcmohtx3_3.insert(1, '*Supplier ID', mid)
zcmohtx3_3.insert(3,'*Zebra Site','TX3')
zcmohtx3_3.insert(4,'Location Code','')
mid = zcmohtx3_3['*Extraction Date']
zcmohtx3_3.drop(labels=['*Extraction Date'], axis=1, inplace = True)
zcmohtx3_3.insert(6, '*Extraction Date', mid)
zcmohtx3_3['Flex String PIT 1']=''
zcmohtx3_3['Flex String PIT 2']=''
zcmohtx3_3['Flex String PIT 3'] = ''
zcmohtx3_3['*Extraction Date'] = zcmohtx3_3['*Extraction Date'].astype(str)
zcmohtx3_3['*Extraction Date'] = zcmohtx3_3['*Extraction Date'] + 'T00:00:00+0000'
zcmohtx3_3

zcmohtx3_3= zcmohtx3_3[(zcmohtx3_3['*Transaction Type']!='BOOKEDSODLY')]
zcmohtx3_3

stringstx3R= 'OH_TX3_DAILY_' + strings 
b=("%s.xlsx" % stringstx3R)

with pd.ExcelWriter(b) as writer:  
       zcmohtx3_3.to_excel(writer, index= False)


#stringstx3= 'OH_TX3_DAILY VALIDATION' + strings 
#b=("%s.xlsx" % stringstx3)

#with pd.ExcelWriter(b) as writer:  
 #      zcmohtx3_1.to_excel(writer, index= False)

#calculate OHTX4
zcmohtx4_1 = zcmohtx2_11.copy()
zcmohtx4_1.drop(labels=['PTO_KIT'], axis=1, inplace = True)
zcmohtx4_1['OH_TX4_DAILY']=zcmohtx4_1['currentOH']-zcmohtx4_1['*Inventory Level']
zcmohtx4_1

#all PN with a measure diff to OHDAILY
zcm_other_measurestx2_1=zcm_other_measurestx2[['*Zebra PN','*Supplier ID']]
zcm_other_measurestx2_1=zcm_other_measurestx2_1.drop_duplicates() 
zcm_other_measurestx2_1

#Which of those rows have not been considered for OHTX4
#merge with OHTX4
zcmohtx4_2 = pd.merge(zcm_other_measurestx2_1,zcmohtx4_1, on =['*Zebra PN','*Supplier ID'],how='left')
zcmohtx4_2
#subset of elements that are not considered
zcmohtx4_3=zcmohtx4_2[['*Zebra PN','*Supplier ID','OH_TX4_DAILY']]
zcmohtx4_3 =zcmohtx4_3[zcmohtx4_3['OH_TX4_DAILY'].isna()]
zcmohtx4_3

zcmohtx4_3['OH_TX4_DAILY']=zcmohtx4_3['OH_TX4_DAILY'].fillna(0)
zcmohtx4_3

#apend those elements to main tab of OH
zcmohtx4_3 = zcmohtx4_1.append(zcmohtx4_3, ignore_index=True)
zcmohtx4_3

zcmohtx4_3.drop(zcmohtx4_3.columns.difference(['*Zebra PN','*Supplier ID','OH_TX4_DAILY']), 1, inplace=True)
zcmohtx4_3

zcmohtx4_3.rename(columns={'OH_TX4_DAILY': '*Inventory Level'},inplace=True)

zcmohtx4_3.insert(0, '#*Zebra', 'Zebra')
mid = zcmohtx4_3['*Supplier ID']
zcmohtx4_3.drop(labels=['*Supplier ID'], axis=1, inplace = True)
zcmohtx4_3.insert(1, '*Supplier ID', mid)
zcmohtx4_3.insert(3,'*Zebra Site','TX4')
zcmohtx4_3.insert(4,'Location Code','')

zcmohtx4_3['*Extraction Date'] = date.today() + timedelta(days=1)
zcmohtx4_3['*Transaction Type'] = 'ONHANDDLY'
zcmohtx4_3['Flex String PIT 1']=''
zcmohtx4_3['Flex String PIT 2']=''
zcmohtx4_3['Flex String PIT 3'] = ''
zcmohtx4_3['*Extraction Date'] = zcmohtx4_3['*Extraction Date'].astype(str)
zcmohtx4_3['*Extraction Date'] = zcmohtx4_3['*Extraction Date'] + 'T00:00:00+0000'
zcmohtx4_3

oh_to_addtx4= pd.read_excel(a,dtype={'*Zebra PN': str, '*Zebra Site': str,'*Supplier ID': str}, sheet_name='TX4')
oh_to_addtx4= oh_to_addtx4[(oh_to_addtx4['#TRANSACTION_TYPE']!='BOOKEDSODLY')]
oh_to_addtx4=oh_to_addtx4[['*Supplier ID','*Zebra PN','#QUANTITY','#TRANSACTION_TYPE','#SCHEDULE_DOCK_DATE']]
oh_to_addtx4.rename(columns={'#TRANSACTION_TYPE': '*Transaction Type','#SCHEDULE_DOCK_DATE':'*Extraction Date'},inplace=True)
#print(len(oh_to_addtx4),'oh_input_file_from_tx4')

#make a df with oh transaction to be summed up id already considered from TX2, or added row if have not consider before
oh_to_addtx4_to_merge= oh_to_addtx4[(oh_to_addtx4['*Transaction Type']=='ONHANDDLY')]
oh_to_addtx4_to_merge.drop(labels=['*Extraction Date'], axis=1, inplace = True)
#print(len(oh_to_addtx4_to_merge), 'current OH in TX4')
oh_to_addtx4_to_merge

#OUTER MERGE as there could be PN on OHTX4 that were not in OHTX2 or PN in new OHTX2 not in current OHTX4
zcmohtx4_5 = pd.merge(oh_to_addtx4_to_merge,zcmohtx4_3, on =['*Zebra PN','*Supplier ID','*Transaction Type'],how='outer')
#print(len(zcmohtx4_5),'oh_from_tx2+oh_from_tx4')
#zcmohtx4_5

zcmohtx4_5['#QUANTITY']= zcmohtx4_5['#QUANTITY'].fillna(0)
zcmohtx4_5['*Inventory Level']= zcmohtx4_5['*Inventory Level'].fillna(0)
#sum current OHTX4 plus new OHTX4 
zcmohtx4_5['*Inventory Level']= zcmohtx4_5['#QUANTITY'] + zcmohtx4_5['*Inventory Level']
zcmohtx4_5

#copy zebra name on NA genereted
zcmohtx4_5.fillna(method='ffill',inplace =True)
zcmohtx4_5

zcmohtx4_5.drop(labels=['#QUANTITY'], axis=1, inplace = True)
zcmohtx4_5

#make a df for measures we will not move
oh_to_addtx4_to_append= oh_to_addtx4[(oh_to_addtx4['*Transaction Type']!='ONHANDDLY')]
oh_to_addtx4_to_append.rename(columns={'#QUANTITY': '*Inventory Level'},inplace=True)
print(len(oh_to_addtx4_to_append))
oh_to_addtx4_to_append

zcmohtx4_4 = zcmohtx4_5.append(oh_to_addtx4_to_append, ignore_index=True)
zcmohtx4_4.fillna(method='ffill',inplace =True)
print(len(zcmohtx4_5), 'lenght before appending (OH_from_TX2+Old_OH_TX4)')
print(len(zcmohtx4_4),'lenght after')

#arrange order of columns
mid = zcmohtx4_4['*Zebra PN']
zcmohtx4_4.drop(labels=['*Zebra PN'], axis=1, inplace = True)
zcmohtx4_4.insert(3, '*Zebra PN', mid)

mid = zcmohtx4_4['*Supplier ID']
zcmohtx4_4.drop(labels=['*Supplier ID'], axis=1, inplace = True)
zcmohtx4_4.insert(2, '*Supplier ID', mid)

mid = zcmohtx4_4['*Transaction Type']
zcmohtx4_4.drop(labels=['*Transaction Type'], axis=1, inplace = True)
zcmohtx4_4.insert(7, '*Transaction Type', mid)

#strings = date.today() + timedelta(days=1)
#strings = str(strings)
stringstx4= 'OH_TX4_DAILY ' + strings 
b=("%s.xlsx" % stringstx4)

with pd.ExcelWriter(b) as writer:  
       zcmohtx4_4.to_excel(writer, index= False)

zcmohtx5_1 = zcmohtx3_1.copy()
zcmohtx5_1['OH_TX5_DAILY']=zcmohtx5_1['currentOH']-zcmohtx5_1['*Inventory Level']
zcmohtx5_1

#all PN with a measure diff to OHDAILY
zcm_other_measurestx3_1=zcm_other_measurestx3[['*Zebra PN','*Supplier ID']]
zcm_other_measurestx3_1=zcm_other_measurestx3_1.drop_duplicates() 
zcm_other_measurestx3_1

#Which of those rows have not been considered for OHTX5
#merge with OHTX5
zcmohtx5_2 = pd.merge(zcm_other_measurestx3_1,zcmohtx5_1, on =['*Zebra PN','*Supplier ID'],how='left')
zcmohtx5_2
#subset of elements that are not considered
zcmohtx5_3=zcmohtx5_2[['*Zebra PN','*Supplier ID','OH_TX5_DAILY']]
zcmohtx5_3 =zcmohtx5_3[zcmohtx5_3['OH_TX5_DAILY'].isna()]
zcmohtx5_3

zcmohtx5_3['OH_TX5_DAILY']=zcmohtx5_3['OH_TX5_DAILY'].fillna(0)
zcmohtx5_3

#apend those elements to main tab of OH
zcmohtx5_3 = zcmohtx5_1.append(zcmohtx5_3, ignore_index=True)
zcmohtx5_3

zcmohtx5_3.drop(zcmohtx5_3.columns.difference(['*Zebra PN','*Supplier ID','OH_TX5_DAILY']), 1, inplace=True)
zcmohtx5_3

zcmohtx5_3.rename(columns={'OH_TX5_DAILY': '*Inventory Level'},inplace=True)

zcmohtx5_3.insert(0, '#*Zebra', 'Zebra')
mid = zcmohtx5_3['*Supplier ID']
zcmohtx5_3.drop(labels=['*Supplier ID'], axis=1, inplace = True)
zcmohtx5_3.insert(1, '*Supplier ID', mid)
zcmohtx5_3.insert(3,'*Zebra Site','TX5')
zcmohtx5_3.insert(4,'Location Code','')

zcmohtx5_3['*Extraction Date'] = date.today() + timedelta(days=1)
zcmohtx5_3['*Transaction Type'] = 'ONHANDDLY'
zcmohtx5_3['Flex String PIT 1']=''
zcmohtx5_3['Flex String PIT 2']=''
zcmohtx5_3['Flex String PIT 3'] = ''
zcmohtx5_3['*Extraction Date'] = zcmohtx5_3['*Extraction Date'].astype(str)
zcmohtx5_3['*Extraction Date'] = zcmohtx5_3['*Extraction Date'] + 'T00:00:00+0000'
zcmohtx5_3

oh_to_addtx5= pd.read_excel(a,dtype={'*Zebra PN': str, '*Zebra Site': str,'*Supplier ID': str}, sheet_name='TX5')
oh_to_addtx5= oh_to_addtx5[(oh_to_addtx5['#TRANSACTION_TYPE']!='BOOKEDSODLY')]
oh_to_addtx5=oh_to_addtx5[['*Supplier ID','*Zebra PN','#QUANTITY','#TRANSACTION_TYPE','#SCHEDULE_DOCK_DATE']]
oh_to_addtx5.rename(columns={'#TRANSACTION_TYPE': '*Transaction Type','#SCHEDULE_DOCK_DATE':'*Extraction Date'},inplace=True)
print(len(oh_to_addtx5),'rows in oh_input_file_from_tx5')

#make a df with oh transaction to be summed up if already considered from TX3, or add row if have not consider before
oh_to_addtx5_to_merge= oh_to_addtx5[(oh_to_addtx5['*Transaction Type']=='ONHANDDLY')]
oh_to_addtx5_to_merge.drop(labels=['*Extraction Date'], axis=1, inplace = True)
print(len(oh_to_addtx5_to_merge))
oh_to_addtx5_to_merge

zcmohtx5_5 = pd.merge(oh_to_addtx5_to_merge,zcmohtx5_3, on =['*Zebra PN','*Supplier ID','*Transaction Type'],how='outer')
print(len(zcmohtx5_5),'oh_from_tx3+oh_from_tx5')
#zcmohtx5_5

zcmohtx5_5['#QUANTITY']= zcmohtx5_5['#QUANTITY'].fillna(0)
zcmohtx5_5['*Inventory Level']= zcmohtx5_5['*Inventory Level'].fillna(0)
zcmohtx5_5['*Inventory Level']= zcmohtx5_5['#QUANTITY'] + zcmohtx5_5['*Inventory Level']
zcmohtx5_5

zcmohtx5_5.fillna(method='ffill',inplace =True)

zcmohtx5_5.drop(labels=['#QUANTITY'], axis=1, inplace = True)
zcmohtx5_5

#make a df for measures we will not move
oh_to_addtx5_to_append= oh_to_addtx5[(oh_to_addtx5['*Transaction Type']!='ONHANDDLY')]
oh_to_addtx5_to_append.rename(columns={'#QUANTITY': '*Inventory Level'},inplace=True)
print(len(oh_to_addtx5_to_append),'rows to append')
oh_to_addtx5_to_append

zcmohtx5_4 = zcmohtx5_5.append(oh_to_addtx5_to_append, ignore_index=True)
zcmohtx5_4.fillna(method='ffill',inplace =True)
print(len(zcmohtx5_5), 'lenght before appending to OH_from_TX3+Old_OH_TX5')
print(len(zcmohtx5_4),'total rows OH_DAILY_TX5')
#arrange order of columns
mid = zcmohtx5_4['*Zebra PN']
zcmohtx5_4.drop(labels=['*Zebra PN'], axis=1, inplace = True)
zcmohtx5_4.insert(3, '*Zebra PN', mid)

mid = zcmohtx5_4['*Supplier ID']
zcmohtx5_4.drop(labels=['*Supplier ID'], axis=1, inplace = True)
zcmohtx5_4.insert(2, '*Supplier ID', mid)

mid = zcmohtx5_4['*Transaction Type']
zcmohtx5_4.drop(labels=['*Transaction Type'], axis=1, inplace = True)
zcmohtx5_4.insert(7, '*Transaction Type', mid)

stringstx5= 'OH_TX5_DAILY ' + strings 
b=("%s.xlsx" % stringstx5)

with pd.ExcelWriter(b) as writer:  
       zcmohtx5_4.to_excel(writer, index= False)

C:\Users\MJ7255\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
<ipython-input-2-c72ac468eba0>:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zcmohtx2_2['*Transaction Type'] = 'ONHANDDLY'
<ipython-input-2-c72ac468eba0>:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zcmohtx2_2['*Extraction Date'] = date.today() + timedelta(days=1)
C:\Users\M

1423
6803 lenght before appending (OH_from_TX2+Old_OH_TX4)
8226 lenght after
2688 rows in oh_input_file_from_tx5
992
4854 oh_from_tx3+oh_from_tx5
1696 rows to append
4854 lenght before appending to OH_from_TX3+Old_OH_TX5
6550 total rows OH_DAILY_TX5


# Write validations for TX4/TX5

In [8]:
#VALIDATIONS WORKING FILE
stringstx4W= 'OH_TX4_DAILY_VALIDATION ' + strings 
b=("%s.xlsx" % stringstx4W)

with pd.ExcelWriter(b) as writer:  
       zcmohtx4_1.to_excel(writer, index= False)

In [9]:
#VALIDATIONS WORKING FILE
stringstx5W= 'OH_TX5_DAILY_VALIDATION ' + strings 
b=("%s.xlsx" % stringstx5W)

with pd.ExcelWriter(b) as writer:  
       zcmohtx5_1.to_excel(writer, index= False)

#TX4 OH will be available from 8 jul, too add to oh (tx2 after july)